# Hematopoiesis LT hidden variable test

In this notebook, we'll perform the test for hidden variables in LT-seq data from hematopoietic stem cells (Fig 4).

We will estimate LMI between day 2 cells and their day 6 progenitors, and compare this to SIMI between sisters cells in separate wells in day 6.

We will perform the estimates for 20 different random pairings of cells within clones (since many clones are sampled more than once at a time point). This will give a sense of the variance of the estimate.

For each estimate, we will shuffle the datasets (unpairing the variables) and estimate LMI as a negative control.


First, we load the libraries and relevant data.


In [1]:
%%capture

from lmi import lmi

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams

from tqdm.notebook import tqdm

import scanpy as sc
from scipy.stats import sem

In [2]:
adata = sc.read('../data/LARRY_HVGs_preprocessed.h5ad')
t2_clones = adata[adata.obs['time'] == 2]
t6_clones = adata[adata.obs['time'] == 6]
common_clones = set([x for x in t2_clones.obs['clone'].values 
                     if x in t6_clones.obs['clone'].values])
print("Number of common clones: %d" %(len(common_clones)))

cxf = adata.to_df()
for name in adata.obs.keys():
    cxf[name] = adata.obs[name].values
rnas = cxf.keys()[:-7]

Number of common clones: 1243


Now we'll use the pandas sample function to randomly pair cells within clones between time points 2 and 6. Then we'll compute SIMI, then shuffle the DataFrames and recompute for the negative control.

In [3]:
N_samples = 20

d = {
    "Shuffled?" : [],
    "Timepoints" : [],
    "LMI" : []
}

for _ in tqdm(range(N_samples)):
    t2 = cxf[(cxf['time'] == 2) & (cxf['clone'].isin(common_clones))].groupby('clone').sample(n=1)
    t6 = cxf[(cxf['time'] == 6) & (cxf['clone'].isin(common_clones))].groupby('clone').sample(n=1)
    pMIs = lmi.lmi(t2[rnas], t6[rnas],)[0]
    
    d["Shuffled?"].append("Real")
    d["Timepoints"].append("2-6")
    d["LMI"].append(np.nanmean(pMIs))
    

    pMIs = lmi.lmi(t2[rnas], t6[rnas].sample(frac=1), epochs=10**3)[0]
    
    d["Shuffled?"].append("Shuffled")
    d["Timepoints"].append("2-6")
    d["LMI"].append(np.nanmean(pMIs))

  0%|          | 0/20 [00:00<?, ?it/s]

In [4]:
df = pd.DataFrame(d)
r = df[df['Shuffled?'] == "Real"]['LMI']
s = df[df['Shuffled?'] == "Shuffled"]['LMI']
print("Mean: %f" % np.mean(r))
print("SEM: %f" % sem(r))
print("Mean: %f" % np.mean(s))
print("SEM: %f" % sem(s))

Mean: 0.299351
SEM: 0.025012
Mean: -0.010936
SEM: 0.006783


In [5]:
w2_clones = t6_clones[t6_clones.obs['well'] == 2]
w1_clones = t6_clones[t6_clones.obs['well'] == 1]
common_clones = set([x for x in w2_clones.obs['clone'].values 
                     if x in w1_clones.obs['clone'].values])
print("Number of common clones: %d" %(len(common_clones)))

Number of common clones: 677


In [6]:
for _ in tqdm(range(N_samples)):

    w2 = cxf[(cxf['well'] == 2) & (cxf['time'] == 6) &
                   (cxf['clone'].isin(common_clones))].groupby('clone').sample(n=1)
    w1 = cxf[(cxf['well'] == 1) & (cxf['time'] == 6) &
                   (cxf['clone'].isin(common_clones))].groupby('clone').sample(n=1)
    

    pMIs = lmi.lmi(w2[rnas], w1[rnas], epochs=10**3)[0]
    
    d["Shuffled?"].append("Real")
    d["Timepoints"].append("6-6")
    d["LMI"].append(np.nanmean(pMIs))
    
    pMIs = lmi.lmi(w2[rnas], w1[rnas].sample(frac=1))[0]
    
    d["Shuffled?"].append("Shuffled")
    d["Timepoints"].append("6-6")
    d["LMI"].append(np.nanmean(pMIs))

  0%|          | 0/20 [00:00<?, ?it/s]

In [7]:
whole_df = pd.DataFrame(d)
df = whole_df[whole_df['Timepoints'] == "6-6"]
r = df[df['Shuffled?'] == "Real"]['LMI']
s = df[df['Shuffled?'] == "Shuffled"]['LMI']
print("Mean: %f" % np.mean(r))
print("SEM: %f" % sem(r))
print("Mean: %f" % np.mean(s))
print("SEM: %f" % sem(s))

Mean: 0.965338
SEM: 0.016375
Mean: 0.000562
SEM: 0.011603


In [8]:
whole_df.groupby(['Timepoints', 'Shuffled?']).mean()

LMI
Timepoints Shuffled?          
2-6        Real       0.299351
           Shuffled  -0.010936
6-6        Real       0.965338
           Shuffled   0.000562

In [9]:
df.to_csv("../results/H_markov_test.csv")